*extract pollution data*



In [1]:
import pandas as pd
import datetime as dt
#  import numpy as np
import json
import requests


# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func


In [2]:
pol_input_file = 'input_data/pollution_us_2000_2016.csv'

In [3]:
pol_df = pd.read_csv(pol_input_file)


In [4]:
# Drop unused columns 
pol_df.drop(columns=['State Code', 'County Code', 'Site Num', 'Address', 'County', 'City', \
                            'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI', \
                            'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', \
                            'SO2 1st Max Value', 'CO 1st Max Hour', 'CO AQI'], inplace=True)

In [5]:
# df.rename(columns={"A": "a", "B": "c"})
pol_df.rename(columns={"Unnamed: 0": "id", "Date Local": "Date_Local", "NO2 Units": "NO2_Units", "NO2 Mean": "NO2_Mean", \
                  "O3 Units": "O3_Units", "O3 Mean": "O3_Mean", "SO2 Units": "SO2_Units", \
                  "SO2 Mean": "SO2_Mean", "CO Units": "CO_Units", "CO Mean": "CO_Mean" }, inplace=True)



In [6]:
pol_df.head()


,id,State,Date_Local,NO2_Units,NO2_Mean,O3_Units,O3_Mean,SO2_Units,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Units,CO_Mean,CO 1st Max Value
0,0,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,21,13.0,Parts per million,1.145833,4.2
1,1,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,21,13.0,Parts per million,0.878947,2.2
2,2,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,2.975000,23,NaN,Parts per million,1.145833,4.2
3,3,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,2.975000,23,NaN,Parts per million,0.878947,2.2
4,4,Arizona,2000-01-02,Parts per billion,22.958333,Parts per million,0.013375,Parts per billion,1.958333,22,4.0,Parts per million,0.850000,1.6


In [7]:
# # iterate/loop over the rows in the DF.  Split 'Date_local', that I assume
# # is MM/DD/YYYY,
# # on forward slash '/' take the third item Year and store in new list 'll_year'
# # then store this in the df as column 'year'
# # for row in df.iterrows():

# # for row in df.iterrows():  NOTE:  Itertion is not suggested trying something different
# #result = [f(x) for x in df['col']]
result = [x.split("-") for x in pol_df['Date_Local']]


In [8]:
# # unpack it
ll_yyyy,ll_mm, ll_dd = map(list, zip(*result))

In [9]:
pol_df['year'] = ll_yyyy
pol_df.head()

,id,State,Date_Local,NO2_Units,NO2_Mean,O3_Units,O3_Mean,SO2_Units,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Units,CO_Mean,CO 1st Max Value,year
0,0,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,21,13.0,Parts per million,1.145833,4.2,2000
1,1,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,21,13.0,Parts per million,0.878947,2.2,2000
2,2,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,2.975000,23,NaN,Parts per million,1.145833,4.2,2000
3,3,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,2.975000,23,NaN,Parts per million,0.878947,2.2,2000
4,4,Arizona,2000-01-02,Parts per billion,22.958333,Parts per million,0.013375,Parts per billion,1.958333,22,4.0,Parts per million,0.850000,1.6,2000


In [10]:
# # pd.to_numeric(s, downcast='integer')
pol_df['year'] = pd.to_numeric(pol_df['year'], downcast='integer')


In [11]:
#import data from url to get the state abbr
url='http://worldpopulationreview.com/static/states/name-abbr.json'
d = requests.get(url).json()
states_abbr = pd.DataFrame(d.items() , columns=["State", "State Abbr"])
states_abbr.head()
states_abbr.rename(columns={"State Abbr": "State_Abbr"}, inplace=True)

In [12]:
pol_updated_df = pd.merge(states_abbr, pol_df, \
                                how='left', on='State')

In [13]:
pol_updated_df.dropna(inplace=True)


pol_13_16_df = pol_updated_df.loc[pol_df['year'] >= 2013]
pol_13_16_df.head()


,State,State_Abbr,id,Date_Local,NO2_Units,NO2_Mean,O3_Units,O3_Mean,SO2_Units,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Units,CO_Mean,CO 1st Max Value,year
1329836,Oregon,OR,80120.0,2010-05-25,Parts per billion,14.916667,Parts per million,0.013625,Parts per billion,1.295833,7.0,3.0,Parts per million,0.362500,0.50,2010.0
1329839,Oregon,OR,80123.0,2010-05-26,Parts per billion,13.583333,Parts per million,0.020167,Parts per billion,1.437500,4.0,6.0,Parts per million,0.327917,0.66,2010.0
1329840,Oregon,OR,80124.0,2010-05-26,Parts per billion,13.583333,Parts per million,0.020167,Parts per billion,1.437500,4.0,6.0,Parts per million,0.366667,0.60,2010.0
1329843,Oregon,OR,80127.0,2010-05-27,Parts per billion,11.000000,Parts per million,0.019250,Parts per billion,1.178261,6.0,1.0,Parts per million,0.315417,0.46,2010.0
1329844,Oregon,OR,80128.0,2010-05-27,Parts per billion,11.000000,Parts per million,0.019250,Parts per billion,1.178261,6.0,1.0,Parts per million,0.345833,0.40,2010.0


# Store this in dir output for loading into the database.
pol_13_16_df.to_csv

In [15]:
pol_13_16_df.to_csv('landing_data/pol_13_16_df.csv', index=False)